In [1]:
from math import exp, sqrt
from random import randrange,choice,random
import os
import numpy as np
from bisect import bisect_right

In [8]:
def find_gt(a, x):
    'Find leftmost value greater than x'
    i = bisect_right(a, x)
    if i != len(a):
        return a[i]
    else: return 1

In [26]:
tmp=sorted(list([0.1,5,32,2,4]))

In [28]:
tmp

[0.1, 2, 4, 5, 32]

In [29]:
find_gt(tmp,32)

1

In [3]:
def init_ising_lattice(n,cool):
    if cool == False:
        lattice = [0]*n
        options = [-1,1]
        for i in range(n):
            lattice[i] = choice(options)
    else:
        lattice=[1]*n

    return lattice

In [4]:
  
def cum_prob(summ):
    return 1-np.exp(-2*summ)

def tool(beta,alpha,d):
    return beta*(d**alpha)

def lookUpTable(L,beta,alpha):
    #define lookup table
    lUpTab=[]
    summa=0
    for dist in range(1,L+1):
        summa+=tool(beta,alpha,dist)
        Cprob=cum_prob(summa)
        lUpTab.append(Cprob)
    return lUpTab

def cluster_for_a_node(n,i,lookup,ising,tempCluster):
    distanceToEnd=n-i
    distanceToBeg=i
    rescaling=1
    Cd=0
    distance=0
    while (distanceToEnd>0):
        g=random()*rescaling+Cd
            #if g >= lookup[-1]: break
        
        Cd=find_gt(lookup,g)
        #print Cd
        distance=bisect_right(lookup, g)
            #print distance
        if i+distance >= n: break
        if ising[i] == ising[i+distance] and i+distance not in tempCluster:
            tempCluster.append(i+distance)
            distanceToEnd=n-i-distance
            #outOfCluster.remove(i+distance)
            #search.remove(i+distance)
        rescaling=1-Cd
        
    
    distance=0
    rescaling=1
    Cd=0
    while (distanceToBeg>0):
        g=random()*rescaling+Cd
            #if g >= lookup[-1]: break
        Cd=find_gt(lookup,g)
        distance=bisect_right(lookup, g)
        if i-distance <= 0: break
        if ising[i] == ising[i-distance]and i-distance not in tempCluster:
            tempCluster.append(i-distance)
            distanceToEnd=i-distance
                #outOfCluster.remove(i-distance)
                #search.remove(i-distance)
        rescaling=(1-Cd)
        
    return tempCluster




def clustering(ising,lookup,n):
    
    #outOfCluster=range(n)
    #search=set(outOfCluster)
    clusters=[]
    #while(len(outOfCluster)>(0.9*n)):
    i=choice(range(n))
    #print i
    tempCluster=[i]
    #tempCluster.append(i)
    #outOfCluster.remove(i)
    #    search.remove(i)
        #single cluster
    new=tempCluster
    while(len(new)>0):
        for item in new:
            tempCluster=cluster_for_a_node(n,item,lookup,ising,tempCluster)
        #print "lung clust"+str(len(tempCluster))
        temp=[item for item in tempCluster if item not in new]
        new=temp
            #if len(outOfCluster)==0:break
            
       
    #print "finito cluster"    
    clusters.append(tempCluster)
        
    
    #clusters needs to be a list of lists
    #print len(clusters)
    #print len(clusters[1])
    return clusters

def clusterSwitch(ising,clusters,h_ex):
    #options=[-1,1]  #1/2 probability to switch  
    h=h_ex
    L=len(ising)
                                   
    for cluster in clusters:
        clusterSize=len(cluster)
        clusterImp=clusterSize/float(L)
        #if clusterSize >0.7*L: print clusterSize
        threshold=np.exp(h*clusterImp)/(np.exp(h*clusterImp)+np.exp(-h*clusterImp))
        if random()<=threshold:
            newSign=-1
        else:
            newSign=1
        toSwitchList=cluster
        
        #newSign=choice(options)  #1/2 probability to switch
        for node in toSwitchList:
            ising[node]=newSign
            
    #modify the lattice by randomly switching clusters
    return ising


  

In [5]:
%matplotlib inline

In [36]:
def main():
    import matplotlib.pyplot as plt
    import sys,getopt
    #opts,args = getopt.getopt(sys.argv[1:],'n:s:b:a:c:h:')
    n = (2**12)*100
    print n
    nsteps = 10**2
    beta=0.4
    alpha=-2.5
    h=-0.
    cool=False
    
    tab=lookUpTable(n,beta,alpha)

    ising=init_ising_lattice(n,cool)
    for i in range(nsteps):


        if i%5==0:
            sys.stdout.write("\rstep %i " % (i))
            sys.stdout.flush()
    
                
        clu=clustering(ising,tab,n)
        ising=clusterSwitch(ising,clu,h)

    #print ising
    print np.mean(ising)

    
 
    
if __name__ == '__main__': main()

409600
step 95 0.001220703125
